In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('data/Data.csv')
labels = pd.read_csv('data/Training_set_labels.csv')
test_data = pd.read_csv('data/Test_set_values.csv')
merged_data = pd.concat([data, test_data])
data_len = len(data)
print('Data Length: ',data_len)
test_data_len = len(test_data)
print('Test Data Length: ', test_data_len)
merged_data_len = len(merged_data)
print('Merged data Length: ',merged_data_len)
merged_data.head()

Data Length:  59400
Test Data Length:  14850
Merged data Length:  74250


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
merged_data.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    4504
gps_height                   0
installer                 4532
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 470
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            4155
recorded_by                  0
scheme_management         4846
scheme_name              35258
permit                    3793
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [4]:
# Drop Schem_name feature..... Too Many Null Values
mergerd_data = merged_data.drop(['scheme_name'], axis = 1)

In [5]:
'''     
count = 0
for col in merged_data['amount_tsh']:
    if col == 0:
        count+=1
print(count)        
'''        

"     \ncount = 0\nfor col in merged_data['amount_tsh']:\n    if col == 0:\n        count+=1\nprint(count)        \n"

In [6]:
#drop funder.. Too many Unique Values
#drop public meeting, and permint columns
# Drop Schem_name feature..... Too Many Null Values

merged_data = merged_data.drop(['funder','subvillage','installer','public_meeting', 'permit','scheme_name','payment','extraction_type','extraction_type_group','quality_group','quantity_group','source_type','waterpoint_type_group'], axis = 1)


In [7]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 27 columns):
id                       74250 non-null int64
amount_tsh               74250 non-null float64
date_recorded            74250 non-null object
gps_height               74250 non-null int64
longitude                74250 non-null float64
latitude                 74250 non-null float64
wpt_name                 74250 non-null object
num_private              74250 non-null int64
basin                    74250 non-null object
region                   74250 non-null object
region_code              74250 non-null int64
district_code            74250 non-null int64
lga                      74250 non-null object
ward                     74250 non-null object
population               74250 non-null int64
recorded_by              74250 non-null object
scheme_management        69404 non-null object
construction_year        74250 non-null int64
extraction_type_class    74250 non-null object
ma

In [8]:
merged_data['wpt_name'].value_counts()

none                     4440
Shuleni                  2183
Zahanati                 1034
Msikitini                 647
Kanisani                  390
Sokoni                    328
Bombani                   323
Ofisini                   321
School                    260
Shule Ya Msingi           247
Shule                     191
Sekondari                 178
Muungano                  159
Mkombozi                  145
Upendo                    130
Madukani                  128
Kituo Cha Afya            113
Mbugani                   113
Umoja                     105
Mkuyuni                   103
Hospital                  100
Center                    100
Kisimani                   97
Ccm                        95
Mtakuja                    91
Songambele                 90
Ofisi Ya Kijiji            89
Bwawani                    86
Tankini                    82
Bondeni                    80
                         ... 
Kwalekidemi                 1
Songambe                    1
Mwamakomel

In [9]:
merged_data.drop(['wpt_name'], axis = 1, inplace =True)

In [10]:
merged_data['date_recorded'] = pd.to_datetime(merged_data['date_recorded'])
merged_data['construction_year'] = merged_data.construction_year

In [11]:
merged_data['operation_year'] = merged_data.date_recorded.dt.year - merged_data.construction_year
merged_data['operation_year'].isnull().sum()

0

In [12]:
merged_data.drop(['date_recorded', 'construction_year','lga','ward','recorded_by','longitude','latitude'], axis = 1, inplace = True)

In [13]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 20 columns):
id                       74250 non-null int64
amount_tsh               74250 non-null float64
gps_height               74250 non-null int64
num_private              74250 non-null int64
basin                    74250 non-null object
region                   74250 non-null object
region_code              74250 non-null int64
district_code            74250 non-null int64
population               74250 non-null int64
scheme_management        69404 non-null object
extraction_type_class    74250 non-null object
management               74250 non-null object
management_group         74250 non-null object
payment_type             74250 non-null object
water_quality            74250 non-null object
quantity                 74250 non-null object
source                   74250 non-null object
source_class             74250 non-null object
waterpoint_type          74250 non-null object
ope

In [14]:
cleaned_train_data = merged_data[:data_len]
cleaned_test_data = merged_data[data_len:]
print('data',data_len)
print('cleaned_train_data',len(cleaned_train_data))
print('test data', test_data_len)
print('cleaned_test_data',len(cleaned_test_data))
cleaned_train_data.to_csv('Cleaned Train Data.csv', index = False)
cleaned_test_data.to_csv('Cleaned Test Data.csv', index =False)

data 59400
cleaned_train_data 59400
test data 14850
cleaned_test_data 14850


In [15]:
merged_data.shape

(74250, 20)

In [16]:
#Collect Categorical Data
cat_data = merged_data.select_dtypes(include=['object'])

In [17]:
cat_data.shape

(74250, 12)

In [18]:
#Collect Numerical Data

num_data = merged_data.drop(cat_data, axis = 1)
num_data.shape

(74250, 8)

In [19]:
num_data.head()

,id,amount_tsh,gps_height,num_private,region_code,district_code,population,operation_year
0,69572,6000.0,1390,0,11,5,109,12
1,8776,0.0,1399,0,20,2,280,3
2,34310,25.0,686,0,21,4,250,4
3,67743,0.0,263,0,90,63,58,27
4,19728,0.0,0,0,18,1,0,2011


In [20]:
dummy = pd.get_dummies(cat_data)

In [21]:
X = pd.concat([num_data, dummy], axis = 1)

In [22]:
X.head()

,id,amount_tsh,gps_height,num_private,region_code,district_code,population,operation_year,basin_Internal,basin_Lake Nyasa,...,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
0,69572,6000.0,1390,0,11,5,109,12,0,1,...,1,0,0,0,1,0,0,0,0,0
1,8776,0.0,1399,0,20,2,280,3,0,0,...,0,1,0,0,1,0,0,0,0,0
2,34310,25.0,686,0,21,4,250,4,0,0,...,0,1,0,0,0,1,0,0,0,0
3,67743,0.0,263,0,90,63,58,27,0,0,...,1,0,0,0,0,1,0,0,0,0
4,19728,0.0,0,0,18,1,0,2011,0,0,...,0,1,0,0,1,0,0,0,0,0


In [23]:
x = X[:data_len]
test_data = X[data_len:]

In [24]:
label = labels.drop(['id'], axis = 1)
label.head()
value = {'functional':2, 'functional needs repair':1,
                   'non functional':0}

label['status_group']  = label.status_group.replace(value)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, label, test_size= 0.20, random_state = 0)

In [26]:
'''
labelencoder = LabelEncoder()
label[:,0] = labelencoder.fit_transform(label[:,0])
onehotencoding = OneHotEncoder(categorical_features = [0])
target_labels = onehotencoding.fit_transform(label)
'''

'\nlabelencoder = LabelEncoder()\nlabel[:,0] = labelencoder.fit_transform(label[:,0])\nonehotencoding = OneHotEncoder(categorical_features = [0])\ntarget_labels = onehotencoding.fit_transform(label)\n'

In [27]:
model = RandomForestClassifier(n_estimators=10)
model.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
scores = model.score(x_test, y_test)
scores

0.76144781144781148

In [29]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Model Accuracy {}'.format(round(accuracy * 100)))

Model Accuracy 76.0


In [30]:
test_data_prediction = model.predict(test_data)
test_data_prediction

array([0, 2, 2, ..., 2, 2, 0], dtype=int64)

In [31]:
#Create submission file

submit = pd.DataFrame({'ID': test_data['id'],
                       'status_group': test_data_prediction
                      })

value = {2:'functional', 1:'functional needs repair',
                   0:'non functional'}

submit['status_group'] = submit.status_group.replace(value)        

submit.to_csv('submission.csv', index=False)